In [1]:
from utils import *
import numpy as np
import json
import os

In [29]:
res_file = r"eval_res\llama3Ins-arc-fullspan-w_exp.json"
if 'arc' in res_file:
    task = "arc_c"
else:
    task = 'it does not matter'

file_name = os.path.basename(res_file).split('.')[0]
print(file_name)
file_name = file_name.replace('_', '-')
file_name_ls = file_name.split('-')
model_name = file_name_ls[0]
dataset_name = file_name_ls[1]

print(f"Model: {model_name}")
print(f"Dataset: {dataset_name}")

llama3Ins-arc-fullspan-w_exp
Model: llama3Ins
Dataset: arc


In [7]:
def preprocess_input_data(dataset, task=None):
    new_data = []
    
    if task in TASK_INST:
        instruction = TASK_INST[task]
    else:
        instruction = None
    for item in dataset:
        if task == "arc_c":
            choices = item["choices"]
            answer_labels = {}
            for i in range(len(choices["label"])):
                answer_key = choices["label"][i]
                text = choices["text"][i]
                if answer_key == "1":
                    answer_labels["A"] = text
                if answer_key == "2":
                    answer_labels["B"] = text
                if answer_key == "3":
                    answer_labels["C"] = text
                if answer_key == "4":
                    answer_labels["D"] = text
                if answer_key in ["A", "B", "C", "D"]:
                    answer_labels[answer_key] = text

            if "D" not in answer_labels:
                answer_labels["D"] = ""
            choices = "\nA: {0}\nB: {1}\nC: {2}\nD: {3}".format(
                answer_labels["A"], answer_labels["B"], answer_labels["C"], answer_labels["D"])
            if "E" in answer_labels:
                choices += "\nE: {}".format(answer_labels["E"])
            item["instruction"] = item["question"] + choices
            item["answers"] = [item["answerKey"]]
        else:
            item["instruction"] = item["question"]
        item["instruction"] = instruction + "\n\n" + item["instruction"] if instruction is not None else item["instruction"]
        new_data.append(item)

    return new_data

In [25]:
res = load_file(res_file)
res = res['results']
data = get_eval_data(res_file)
data = preprocess_input_data(data, task=task)
assert len(res) == len(data)

In [10]:
res[0].keys()

dict_keys(['no_retrieval', 'no_retrieval_ids', 'no_retrieval_log_probs', 'all_doc_retrieval', 'all_doc_retrieval_ids', 'all_doc_retrieval_log_probs', 'retrieval', 'retrieval_token_ids', 'retrieval_log_probs', 'retrieval_res', 'question_id', 'gold'])

In [11]:
res[0]['retrieval_res']['retrieval_0'].keys()

dict_keys(['pred', 'score', 'id_log_probs', 'token_ids'])

In [26]:
new_res = []

In [27]:
for ind, item in enumerate(res):
    data_item = data[ind]
    question = data_item['question']
    item_ = item.copy()
    item_['question'] = question
    for i in range(5):
        item_[f'ret_{i}'] = item['retrieval_res'][f'retrieval_{i}']['pred']
        item_[f'ret_{i}_log_probs'] = item['retrieval_res'][f'retrieval_{i}']['id_log_probs']
        ctx_ = data_item['ctxs'][i]
        item_[f'ret_{i}_ctx'] = f"{ctx_['title']}\n{ctx_['text']}"
        item_[f'ret_{i}_ctx_score'] = ctx_['score']
        item_[f"ret_{i}_scores"] = {"qa": 0, "qc": 0, "ca": 0, "ret_score": ctx_['score']}
        
    item_['all_doc_retrieval_ctx'] = '\n'.join([f"{ctx['title']}\n{ctx['text']}" for ctx in data_item['ctxs'][:5]])
    item_['all_doc_retrieval_ctx_score'] = np.mean([float(ctx['score']) for ctx in data_item['ctxs'][:5]])
    item_['all_doc_retrieval_scores'] = {"qa": 0, "qc": 0, "ca": 0, "ret_score": np.mean([float(ctx['score']) for ctx in data_item['ctxs'][:5]])}
    
    item_["no_retrieval_scores"] = {"qa": 0, "qc": 0, "ca": 0, "ret_score": np.mean([float(ctx['score']) for ctx in data_item['ctxs'][:5]])}
    new_res.append(item_)
    
with open(f"pending_process\{model_name}-{dataset_name}.json", "w") as f:
    json.dump(new_res, f)



In [8]:
a = np.float32(0.0)
np.float64(a)
from functools import singledispatch
import json

@singledispatch
def to_serializable(val):
    """Used by default."""
    return str(val)


@to_serializable.register(np.float32)
def ts_float32(val):
    """Used if *val* is an instance of numpy.float32."""
    return np.float64(val)
with open("test.json", "w") as f:
    json.dump({"a":a}, f, default=to_serializable)

In [10]:
from utils import *
import os
from selfcheckGPT_wrapper import MyNLI
import numpy as np
import json
from dotenv import load_dotenv, find_dotenv
from functools import singledispatch

load_dotenv(find_dotenv())
MODEL_ROOT = os.environ.get("model_path")

model_path = f"{MODEL_ROOT}/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7" #deberta-v3-large-mnli
selfcheck_nli = MyNLI(model_path)
# probs = selfcheck_nli.compute_nli_score(examples)

@singledispatch
def to_serializable(val):
    """Used by default."""
    return str(val)


@to_serializable.register(np.float32)
def ts_float32(val):
    """Used if *val* is an instance of numpy.float32."""
    return np.float64(val)

def process(file_name):
    res = []
    data = load_file(file_name)
    for i, data_item_ in enumerate(data):
        data_item = data_item_.copy()
        pred_types = ["all_doc_retrieval", "no_retrieval", "ret_0", "ret_1", "ret_2", "ret_3", "ret_4"]
        q = data_item["question"]
        for pred_type in pred_types:
            a = data_item[pred_type]
            
            if pred_type != 'no_retrieval':
                c = data_item[pred_type + "_ctx"]
                nil_candidates = [(q, a), (q, c), (c, a)]
            else:
                nil_candidates = [(q, a)]
            nil_scores = selfcheck_nli.compute_nli_score(nil_candidates).cpu().detach().numpy()
            data_item[pred_type + "_scores"]['qa'] = nil_scores[0][0]
            if pred_type != 'no_retrieval':
                data_item[pred_type + "_scores"]['qc'] = nil_scores[1][0]
                data_item[pred_type + "_scores"]['ca'] = nil_scores[2][0]

        data_item["no_retrieval_scores"]['qc'] = np.mean([data_item[f"ret_{i}_scores"]['qc'] for i in range(5)] + [data_item["all_doc_retrieval_scores"]['qc']])
        data_item["no_retrieval_scores"]['ca'] = np.mean([data_item[f"ret_{i}_scores"]['ca'] for i in range(5)] + [data_item["all_doc_retrieval_scores"]['ca']])
        res.append(data_item)


    file_name = os.path.basename(file_name).split('.')[0]
    with open(f"{file_name}_processed_2mil7.json", "w") as f:
        json.dump(res, f, default=to_serializable)
        

if __name__ == "__main__":
    files = ["llama2chat-arc.json", "llama2chat-health.json", "llama3Ins-arc.json", "llama3Ins-health.json", "llama3Ins-pqa.json", "llama3Ins-tqa.json"]
    for f in files:
        process(f'ft_local/{f}')

In [7]:

file_name = "selfrag-arc-fullspan.json"
res_file = fr"eval_res\{file_name}"
eval_data = get_eval_data(res_file)

with open(res_file, "r") as f:
    res_file = json.load(f)
    res = res_file['results']
    
for ind, item in enumerate(res):
    data_item = eval_data[ind]
    question = data_item['question']
    item['question'] = question
    for i in range(5):
        item["retrieval_res"][f"retrieval_{i}"]['ctx_score'] = data_item['ctxs'][i]['score']
res_file['results'] = res        

with open(f"post_processed\{file_name}", "w") as f:
    json.dump(res_file, f)